In [1]:
import numpy as np
import gym
import math
import torch as T
import random
from sac_trader import SAC_Trader

In [2]:
s0 = 100 # initial mid-price

T = 1 # terminal time

sigma = 2 # volatility

dt = 0.005 # time step

N = 200

gamma = 0.1 # risk aversion parameter

k = 1.5 # k = alpha*K and this is determined by the market order size

A = 140 # scaling factor that depends on frequency  of market buy or sell orders and market order size

In [3]:
# Simulate geometric Brownian motion
s = np.zeros(N)
s[0] = s0
W = np.random.normal(0, np.sqrt(dt), N)
for i in range(1, N):
    ds = sigma * W[i-1]
    s[i] = s[i-1] + ds

In [4]:
SAC_Trader = SAC_Trader(s, sigma, A, k, dt, 1000)

AttributeError: 'NoneType' object has no attribute 'action_space'